# Py-EMDE
### Python Email Data Entry

The following code can gather data from weather stations reporting to the [CHORDS](http://ncar.github.io/chords/) portal, package it up into the proper format for [GLOBE Email Data Entry](https://www.globe.gov/globe-data/data-entry/) , and send it using the [SparkPost](https://www.sparkpost.com/) API.

*In order to send email, you'll need to setup SparkPost by creating an account and confirming you own the domain you'll be sending emails from. You'll also need to create a SparkPost API key and set the environment variable SPARKPOST_API_KEY equal to the value of your API key. This script can be further modified to use a different method for sending email if needed.*

This code will contact the CHORDS Portal and collect all the measurement data from the specified instrument, in the specified date range.

In [36]:
import requests
import json


r = requests.get('http://3d-kenya.chordsrt.com/instruments/1.geojson?start=2016-09-01T00:00&end=2016-11-01T00:00')
if r.status_code == 200:
    d = r.json()['Data']
else:
    print("Please verify that the URL for the weather station is correct. You may just have to try again with a different/smaller date range or different dates.")

Now the collected data can be viewed simply by issuing the following command

In [ ]:
d

This code is useful for looking at a specific measurement dataset

In [ ]:
for o in d:
    if o['variable_shortname'] == 'msl1':
        print(o['time'], o['value'], o['units'])

A modified version of the above code will format the data properly for GLOBE Email Data Entry

In [25]:
davad_tuple = (
    'f1',
    'f2',
    'f3',
    'f4',
    'f5',
    'f6',
    'f7',
    'f8',
    'f9',
    'f10',
    'f11',
    'f12',
    'f13',
    'f14',
)

def make_data_set(d):
    data_list = []
    for o in d:
        if o['variable_shortname'] == 'rain':
            t = o['time'].split("T")
            tdate = t[0].replace('-', '')
            ttime = ''.join(t[1].split(':')[:-1])
            rain = o['value']
            if ttime.endswith('00') or ttime.endswith('15') or ttime.endswith('30') or ttime.endswith('45'):
                davad_tuple = ['DAVAD', 'GLIDGDTR', 'SITE_ID:45015']+['X']*11
                davad_tuple[3] = tdate + ttime
                davad_tuple[11] = str(rain)
                data_list.append('{}'.format(' '.join(davad_tuple)))
    #print('//AA\n{}\n//ZZ'.format('\n'.join(data_list)))
    return data_list

To see the data formatted in GLOBE Email Data Entry format, comment out the return data_list command above, uncomment the print command right above it, then issue the following command

In [ ]:
make_data_set(d)

To email the data set to GLOBE's email data entry server, run the following code.

In [10]:
def email_data(data_list):
    import os
    
    from sparkpost import SparkPost
    
    FROM_EMAIL = os.getenv('FROM_EMAIL')
    BCC_EMAIL = os.getenv('BCC_EMAIL')

    # Send email using the SparkPost api
    sp = SparkPost() # uses environment variable named SPARKPOST_API_KEY

    response = sp.transmission.send(
            recipients=['data@globe.gov'],
            bcc=[BCC_EMAIL],
            text='//AA\n{}\n//ZZ'.format('\n'.join(data_list)),
            from_email=FROM_EMAIL,
            subject='DATA'
    )

    print(response)

Finally, this command sends the email

In [37]:
email_data(make_data_set(d))

{'total_rejected_recipients': 0, 'total_accepted_recipients': 2, 'id': '102663889934893248'}
